In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from tensorflow.keras.layers import Input, Lambda, Dense, Concatenate, Conv1D, GlobalAveragePooling1D, Dropout, BatchNormalization, Bidirectional, LSTM, Layer
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder, normalize
from collections import Counter
import random

# Firebase

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Khởi tạo kết nối với Firebase
cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://coffee-4053c-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [ ]:
def GetProducts():
    # Lấy dữ liệu từ nút "SanPham" trong Firebase
    product_data = db.reference('SanPham').get()

    # Lấy dữ liệu từ nút "LoaiSanPham" trong Firebase
    product_type_data = db.reference('LoaiSanPham').get()
    list_product_type = list(product_type_data.values())

    # Xử lý dữ liệu
    products = [
        {
            'MaSanPham': product['MaSanPham'],
            'TenSanPham': product['TenSanPham'],
            'SoLuong': product['SoLuong'],
            'MoTa': product['Mota'],
            'HinhAnh': product['HinhAnh'],
            'LoaiSanPham': next((x['LoaiSanPham'] for x in list_product_type if x['MaLoaiSanPham'] == product['MaLoaiSanPham']), None),
            'CongThuc': list(product['CongThuc'].values())
        }
        for product in product_data.values()
    ]

    return products

In [ ]:
def GetTopProductID(number = 5):
  bill_data = db.reference('HoaDon').get()

  product_quantities = Counter()

  for bill_details in bill_data.values():
      for product_details in bill_details['ChiTietHoaDon'].values():
          product_quantities[product_details['MaSanPham']] += product_details['SoLuong']

  top_products = product_quantities.most_common(number)

  return [productID for productID, quantity in top_products]

In [ ]:
products = GetProducts()
product_size = len(products)

In [ ]:
products[:5]

[{'MaSanPham': 'SP0001',
  'TenSanPham': 'Cà phê sữa',
  'SoLuong': 11,
  'MoTa': 'Cà phê sữa siêu ngon',
  'HinhAnh': 'https://res.cloudinary.com/dev9hnuhw/image/upload/v1717966721/coffee/vigdytrefgtuwev3wsnx.jpg',
  'LoaiSanPham': 'Cà phê',
  'CongThuc': [{'MaDonVi': 'DV0002',
    'MaNguyenLieu': 'NL0003',
    'SoLuong': 20,
    'TenDonVi': 'g',
    'TenNguyenLieu': 'Cà phê'},
   {'MaDonVi': 'DV0004',
    'MaNguyenLieu': 'NL0006',
    'SoLuong': 200,
    'TenDonVi': 'ml',
    'TenNguyenLieu': 'Sữa'}]},
 {'MaSanPham': 'SP0002',
  'TenSanPham': 'Americano đá',
  'SoLuong': 3,
  'MoTa': 'Americano đá siêu ngon',
  'HinhAnh': 'https://res.cloudinary.com/dev9hnuhw/image/upload/v1715354979/coffee/u1tgayzvetmsbihacuhp.png',
  'LoaiSanPham': 'Cà phê',
  'CongThuc': [{'MaDonVi': 'DV0001',
    'MaNguyenLieu': 'NL0003',
    'SoLuong': 10,
    'TenDonVi': 'g',
    'TenNguyenLieu': 'Cà phê'}]},
 {'MaSanPham': 'SP0003',
  'TenSanPham': 'Latte đá',
  'SoLuong': 9,
  'MoTa': 'Latte đá siêu ngon',
  

In [ ]:
def GetProductRC():
    product_data = db.reference('SanPham').get()

    product_type_data = db.reference('LoaiSanPham').get()
    list_product_type = list(product_type_data.values())

    products = [
        {
            'MaSanPham': product['MaSanPham'],
            'TenSanPham': product['TenSanPham'],
            'SoLuong': product['SoLuong'],
            'LoaiSanPham': next((x['LoaiSanPham'] for x in list_product_type if x['MaLoaiSanPham'] == product['MaLoaiSanPham']), None),
            'CongThuc': ' | '.join([congthuc['TenNguyenLieu'] for congthuc in product['CongThuc'].values()])
        }
        for product in product_data.values()
    ]

    return products

In [ ]:
def GetProductById(productID):
    # Reference to the 'SanPham' node
    product_ref = db.reference('SanPham')
    products_data = product_ref.get()

    # Reference to the 'LoaiSanPham' node
    product_type_data = db.reference('LoaiSanPham').get()
    list_product_type = list(product_type_data.values())

    # Search for the product by ID
    for product_id, product in products_data.items():
        if product.get('MaSanPham') == productID:
            # Format the product data to include 'LoaiSanPham' and 'CongThuc'
            formatted_product = {
                'MaSanPham': product['MaSanPham'],
                'TenSanPham': product['TenSanPham'],
                'SoLuong': product['SoLuong'],
                'LoaiSanPham': next((x['LoaiSanPham'] for x in list_product_type if x['MaLoaiSanPham'] == product['MaLoaiSanPham']), None),
                'CongThuc': ' | '.join([congthuc['TenNguyenLieu'] for congthuc in product['CongThuc'].values()])
            }
            return formatted_product

    return None

In [ ]:
def GetEvaluate():
    evaluate_data = db.reference('DanhGia').get()

    evaluates = [
        {
            'MaKhachHang': evaluete['MaNguoiDung'],
            'MaSanPham': evaluete['MaSanPham'],
            'DiemDanhGia': evaluete['DiemDanhGia']
        }
        for evaluete in evaluate_data.values()
    ]

    return evaluates

In [ ]:
def GetIngredients():
    ingredient_data = db.reference('NguyenLieu').get()
    ingredients = list(ingredient_data.values())

    return ingredients

In [ ]:
ingredients = GetIngredients()

In [ ]:
ingredients

[{'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0001',
  'SoLuong': 34,
  'TenNguyenLieu': 'Muối'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0002',
  'SoLuong': 38,
  'TenNguyenLieu': 'Đường'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0003',
  'SoLuong': 8.8,
  'TenDonVi': 'Kg',
  'TenNguyenLieu': 'Cà phê'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0004',
  'SoLuong': 10,
  'TenDonVi': 'Kg',
  'TenNguyenLieu': 'Ca cao'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0005',
  'SoLuong': 10,
  'TenNguyenLieu': 'Bột quế'},
 {'MaDonVi': 'DV0003',
  'MaNguyenLieu': 'NL0006',
  'SoLuong': 7,
  'TenDonVi': 'l',
  'TenNguyenLieu': 'Sữa'},
 {'MaDonVi': 'DV0003',
  'MaNguyenLieu': 'NL0007',
  'SoLuong': 10,
  'TenNguyenLieu': 'Trà đen'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0008',
  'SoLuong': 10,
  'TenNguyenLieu': 'Cà phê hoà tan'},
 {'MaDonVi': 'DV0004',
  'MaNguyenLieu': 'NL0009',
  'SoLuong': 10000,
  'TenDonVi': 'ml',
  'TenNguyenLieu': 'Kem tươi'},
 {'MaDonVi': 'DV0003',
  'MaNg

# Gợi ý sản phẩm

In [ ]:
class CB(object):
    def __init__(self, Y_data, k=10, bias_factor=0.5):
        """
        Initialize the Content-Based Recommendation model.

        :param Y_data: List of products, each represented as a dictionary of attributes.
        :param k: Number of recommendations to return.
        :param bias_factor: Scaling factor for dynamic bias to control the balance between similarity and randomness.
        """
        self.Y_data = Y_data
        self.k = k
        self.weights = {"LoaiSanPham": 0.4, "CongThuc": 0.6}
        self.bias_factor = bias_factor
        self.tfidf = TfidfVectorizer()
        self.one_hot_encoder = OneHotEncoder(sparse_output=False)

    def embeddings_matrix(self):
        """
        Create the embeddings matrix based on product attributes.
        - LoaiSanPham (Product Type) uses one-hot encoding.
        - CongThuc (Recipe) uses TF-IDF for vectorization.
        """
        type_features = self.one_hot_encoder.fit_transform(
            [[p["LoaiSanPham"]] for p in self.Y_data]
        ) * self.weights["LoaiSanPham"]

        recipe_features = self.tfidf.fit_transform(
            [p["CongThuc"] for p in self.Y_data]
        ).toarray() * self.weights["CongThuc"]

        self.combined_features = normalize(
            np.hstack([type_features, recipe_features])
        )

    def calculate_similarity(self):
        """
        Calculate the similarity matrix with a dynamic bias.
        - Cosine similarity is calculated for the embeddings.
        - A dynamic bias is added to the similarity score to promote diversity.
        """
        base_similarity = cosine_similarity(self.combined_features)

        random_bias = np.random.rand(*base_similarity.shape)

        self.similarity_matrix = base_similarity + self.bias_factor * random_bias * (1 - base_similarity)

    def refresh(self):
        """
        Refresh the embeddings matrix and similarity matrix.
        This should be called whenever the product data changes.
        """
        self.embeddings_matrix()
        self.calculate_similarity()

    def fit(self):
        """
        Initialize and prepare the recommendation model.
        """
        self.refresh()

    def recommend(self, product_id):
        """
        Recommend products based on a given product ID.

        :param product_id: The product ID to base recommendations on.
        :return: A list of recommended products.
        """
        idx = next(i for i, Y in enumerate(self.Y_data) if Y["MaSanPham"] == product_id)

        sorted_indices = sorted(
            enumerate(self.similarity_matrix[idx]),
            key=lambda x: -x[1]
        )

        return [self.Y_data[i] for i, _ in sorted_indices[:self.k]]

In [ ]:
class CF(object):
    def __init__(self, k=10):
        self.user_factors = np.load("user_factors.npy")
        self.item_factors = np.load("item_factors.npy")

        with open("user_index_mapping.pkl", "rb") as f:
            self.user_index_mapping = pickle.load(f)
        with open("item_index_mapping.pkl", "rb") as f:
            self.item_index_mapping = pickle.load(f)

        self.model = load_model("recommendation_model.h5")

        self.k = k

    def recommend(self, u):
        user_index = self.user_index_mapping.get(u)

        if user_index is not None:
            user_vector = self.user_factors[user_index]

            predicted_scores = []

            for item_id, item_index in self.item_index_mapping.items():
                item_vector = self.item_factors[item_index]
                score = self.model.predict([np.array([user_vector]), np.array([item_vector])])[0][0]
                predicted_scores.append((item_id, score))

            predicted_scores.sort(key=lambda x: x[1], reverse=True)

            # print(predicted_scores)

            return [item_id for item_id, score in predicted_scores[:self.k]]

        else:
            return []

In [ ]:
class HybridRecommender:
    def __init__(self, products, k = 10):
        self.cb_recommender = CB(products, k)
        self.cb_recommender.fit()

        self.cf_recommender = CF(k = k)

        self.k = k

        self.products = products

    def recommend(self, user_id, product_id, isCF, isCB):
        recommendations = []

        if user_id and isCF:
            # Sử dụng Collaborative Filtering đầu tiên
            # Chỉ có các mã sản phẩm
            cf_recommendations = self.cf_recommender.recommend(user_id)
            cf_recommendations = [product for product in self.products if product['MaSanPham'] in cf_recommendations]
            recommendations = cf_recommendations

        if product_id and isCB:
            if len(recommendations) < self.k:
                # Nếu không đủ, sử dụng Content-based Filtering
                num_more = self.k - len(recommendations)
                cb_recommendations = self.cb_recommender.recommend(product_id)

                for product in cb_recommendations:
                    if product['MaSanPham'] == product_id:
                        continue

                    if num_more == 0:
                        break

                    if product not in recommendations:
                        recommendations.append(product)
                        num_more -= 1

        return recommendations

In [ ]:
productsRC = GetProductRC()
RS = HybridRecommender(productsRC, k = 10)

In [ ]:
resultRC = RS.recommend("KH0001", "SP0039", False, True)

In [ ]:
def display_products(products):
    print("="*200)
    for product in products:
        print(f"{product['MaSanPham']:<15}{product['TenSanPham']:<70}{product['LoaiSanPham']:<15}")

In [ ]:
print(f"{'Mã Sản Phẩm':<15}{'Tên Sản Phẩm':<70}{'Loại Sản Phẩm':<15}")
display_products([GetProductById("SP0039")])
print()
display_products(resultRC)

Mã Sản Phẩm    Tên Sản Phẩm                                                          Loại Sản Phẩm  
SP0039         Coffee                                                                Cà phê         

Mã Sản Phẩm    Tên Sản Phẩm                                                          Loại Sản Phẩm  
SP0002         Americano đá                                                          Cà phê         
SP0009         Cà phê Espresso                                                       Cà phê         
SP0025         Caramel Frappuccino Blended Coffee                                    Cà phê         
SP0029         Cascara Coconutmilk Latte                                             Cà phê         
SP0034         Cinnamon Dolce Latte                                                  Cà phê         
SP0044         Cupcake Crme Frappuccino Blended Crme                                 Cà phê         
SP0049         Evolution Fresh ColdPressed Apple Berry Juice                         Cà ph

# Chatbot

In [ ]:
# Configuration and Model Parameters
bert_name = "vinai/phobert-base"

# Load pre-trained BERT model and tokenizer
bert = TFAutoModel.from_pretrained(bert_name)
tokenizer = AutoTokenizer.from_pretrained(bert_name)

MAX_LEN_CONTEXT = 25

class BERTEmbeddingLayer(Layer):
    def __init__(self, bert_model_name, max_length, **kwargs):
        super(BERTEmbeddingLayer, self).__init__(**kwargs)
        self.bert_model_name = bert_model_name
        self.max_length = max_length
        # Load the BERT model inside the layer
        self.bert = TFAutoModel.from_pretrained(bert_model_name)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        # Obtain BERT outputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Equivalent to outputs[0]

    def get_config(self):
        config = super(BERTEmbeddingLayer, self).get_config()
        config.update({
            'bert_model_name': self.bert_model_name,
            'max_length': self.max_length,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
model = tf.keras.models.load_model('model_coffee_question_classification.keras', custom_objects={'BERTEmbeddingLayer': BERTEmbeddingLayer})

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
MAX_LEN_CONTEXT = 25

def getNumberPredict(question):
  test_contexts = [question]

  # Tokenize the test contexts
  test_encodings = tokenizer(
      test_contexts,
      truncation=True,
      padding='max_length',
      max_length=MAX_LEN_CONTEXT,
      add_special_tokens=True,
      return_tensors='tf'
  )

  # Prepare input tensors
  test_input_ids = test_encodings['input_ids']
  test_attention_mask = test_encodings['attention_mask']

  # Make predictions using the model
  predictions = model.predict([test_input_ids, test_attention_mask], batch_size=32)

  # Convert predictions to class labels (e.g., using argmax)
  predicted_classes = np.argmax(predictions, axis=1)

  return predicted_classes[0]

  # If you have a dictionary mapping from class indices to class labels, apply it
  # Example:
  label_mapping = {0: 'infomation', 1: 'productTopSell', 2: 'recommendedProduct', 3: 'ingredient', 4: 'quality', 5: 'greeting'}
  predicted_labels = [label_mapping.get(pred, 'unknown') for pred in predicted_classes]

  # Print the predicted labels
  for context, label in zip(test_contexts, predicted_labels):
      print(f"Question: {context} --> Predicted Label: {label}")

# DSA

In [ ]:
# KMP algorithm for searching product names in a precomputed LPS of a question
def compute_lps_array(pattern):
    lps = [0] * len(pattern)
    length = 0  # Length of the previous longest prefix suffix
    i = 1

    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1

    return lps

def kmp_search_with_precomputed_lps(text, pattern, lps):
    i = 0  # Index for text
    j = 0  # Index for pattern

    while i < len(text):
        if pattern[j] == text[i]:
            i += 1
            j += 1

        if j == len(pattern):
            return True  # Pattern found
        elif i < len(text) and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1

    return False  # Pattern not found

# Function to find product names in a question using precomputed LPS
def find_products_in_question_kmp_precomputed(question, product_list):
    question_lower = question.lower()
    lps_question = compute_lps_array(question_lower)

    detected_products = []

    for product in product_list:
        product_name = product["TenSanPham"].lower()
        if kmp_search_with_precomputed_lps(question_lower, product_name, lps_question):
            detected_products.append(product)

    return detected_products

def find_ingredients_in_question_kmp_precomputed(question, ingredient_list):
    question_lower = question.lower()
    lps_question = compute_lps_array(question_lower)

    detected_ingredients = []

    for ingredient in ingredient_list:
        ingredient_name = ingredient["TenNguyenLieu"].lower()
        if kmp_search_with_precomputed_lps(question_lower, ingredient_name, lps_question):
            detected_ingredients.append(ingredient)

    return detected_ingredients

In [ ]:
def getProductFromIngredients(products, ingredients):
    ingredient_codes = {ingredient["MaNguyenLieu"] for ingredient in ingredients}

    matching_products = []

    for product in products:
        product_ingredient_codes = {ingredient['MaNguyenLieu'] for ingredient in product['CongThuc']}

        if ingredient_codes.issubset(product_ingredient_codes):
            matching_products.append(product)

    return matching_products[:5]

In [ ]:
def binary_search(product_id):
    left, right = 0, product_size - 1
    while left <= right:
        mid = (left + right) // 2
        mid_id = extract_number_from_id(products[mid]['MaSanPham'])
        target_id = extract_number_from_id(product_id)
        if mid_id == target_id:
            return products[mid]
        elif mid_id < target_id:
            left = mid + 1
        else:
            right = mid - 1
    return None  # Không tìm thấy sản phẩm

## Chatbot

In [ ]:
def extract_number_from_id(product_id):
    # Lấy phần số từ mã sản phẩm dạng "SP0001"
    return int(product_id[2:])

In [ ]:
def getInfomationProduct(product):
    # Tạo câu mô tả sản phẩm
    text = f"{product['TenSanPham']} thuộc loại sản phẩm {product['LoaiSanPham']}. "
    text += f"Sản phẩm này được mô tả như sau: {product['MoTa']}. "
    text += "Nguyên liệu chính gồm: "

    # Danh sách các nguyên liệu từ 'CongThuc'
    ingredients = ", ".join(congthuc.get('TenNguyenLieu', 'N/A') for congthuc in product['CongThuc'])
    text += f"{ingredients}. "

    # Thông tin về số lượng sản phẩm hiện có
    text += f"Số lượng hiện tại trong kho: {product['SoLuong']}."

    return text

In [ ]:
import random

def get_quality_info(products):
    product = ""

    if len(products) == 1:
        product = products[0]
    else:
        product = random.choice(products)

    message = f"Sản phẩm {product['TenSanPham']} rất ngon "
    ingredients = ", ".join(congthuc.get('TenNguyenLieu', 'N/A') for congthuc in product['CongThuc'])
    message += f"vì được làm từ những nguyên liệu chất lượng như {ingredients}."

    return message

In [ ]:
beginMessage = "Chào bạn, ESPRO xin trả lời câu hỏi của bạn lưu ý đây là tin nhắn tự động của trí tuệ nhân tạo."
endMessage = (
    "Mong bạn hài lòng với câu trả lời của chúng tôi.\n"
    "Hiện tại, chúng tôi chỉ trả lời các câu hỏi liên quan đến thông tin sản phẩm, "
    "sản phẩm bán chạy và gợi ý sản phẩm.\nVí dụ:\n"
    "- Cà phê sữa còn hàng không?\n"
    "- Cho tôi danh sách sản phẩm bán chạy của quán.\n"
    "- Gợi ý cà phê tốt nhất.\n"
    "Chân thành cảm ơn bạn đã sử dụng dịch vụ của quán."
)

In [ ]:
def getMessage(number, question, customer_id):
    message = [beginMessage]

    # Find products in the question
    productInQuestion = find_products_in_question_kmp_precomputed(question, products)
    sizeProductInQuestion = len(productInQuestion)

    ingredientInQuestion = find_ingredients_in_question_kmp_precomputed(question, ingredients)

    if number == 0:
        # Information
        message.extend(getInfomationProduct(product) for product in productInQuestion)

    elif number == 1:
        # Top sell
        productIDs = GetTopProductID()
        productTopSell = [binary_search(productID) for productID in productIDs]

        message.append('Sản phẩm được bán chạy nhiều nhất của cửa hàng:')
        message.extend(f'- {product["TenSanPham"]}' for product in productTopSell)

    elif number == 2:
        # Recommendation
        product_id = productInQuestion[0]['MaSanPham'] if sizeProductInQuestion != 0 else "SP0001"
        user_id = "" if sizeProductInQuestion != 0 else customer_id

        productsRecommend = RS.recommend(user_id, product_id, True, True)

        message.append('Sản phẩm gợi ý cho bạn:')
        message.extend(f'- {product["TenSanPham"]}' for product in productsRecommend)

    elif number == 3:
        productFromIngredients = getProductFromIngredients(products, ingredientInQuestion)
        message.append('Các sản phẩm có thành phần bạn quan tâm:')
        message.extend(f'- {getInfomationProduct(product)}' for product in productFromIngredients)

    elif number == 4:
        # Quality information
        if productInQuestion:
          message.append(f'Thông tin về chất lượng của sản phẩm: {get_quality_info(productInQuestion)}')
        else:
          message.append('Sản phẩm của cửa hàng đều ngon.')

    elif number == 5:
        # Greeting
        message = ['Chào bạn! Cảm ơn bạn đã quan tâm đến cửa hàng của chúng tôi. Nếu bạn cần thông tin cụ thể về sản phẩm hoặc dịch vụ, vui lòng hỏi nhé!']

    # Combine the messages into a single string
    return '\n'.join(message) + '\n' + endMessage

# API

In [ ]:
def chatBotGetMessage(question, customer_id):
  number = getNumberPredict(question)
  return getMessage(number, question, customer_id)

In [ ]:
print(chatBotGetMessage("tôi nên dùng cà phê nào", "KH0001"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━

In [ ]:
print(chatBotGetMessage("Cà phê sữa còn hàng không", "KH0001"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Chào bạn, ESPRO xin trả lời câu hỏi của bạn lưu ý đây là tin nhắn tự động của trí tuệ nhân tạo.
Cà phê sữa thuộc loại sản phẩm Cà phê. Sản phẩm này được mô tả như sau: Cà phê sữa siêu ngon. Nguyên liệu chính gồm: Cà phê, Sữa. Số lượng hiện tại trong kho: 11.
Mong bạn hài lòng với câu trả lời của chúng tôi.
Hiện tại, chúng tôi chỉ trả lời các câu hỏi liên quan đến thông tin sản phẩm, sản phẩm bán chạy và gợi ý sản phẩm.
Ví dụ:
- Cà phê sữa còn hàng không?
- Cho tôi danh sách sản phẩm bán chạy của quán.
- Gợi ý cà phê tốt nhất.
Chân thành cảm ơn bạn đã sử dụng dịch vụ của quán.


In [ ]:
print(chatBotGetMessage("Cho tôi danh sách sản phẩm bán chạy của quán", "KH0001"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Chào bạn, ESPRO xin trả lời câu hỏi của bạn lưu ý đây là tin nhắn tự động của trí tuệ nhân tạo.
Sản phẩm được bán chạy nhiều nhất của cửa hàng:
- Americano đá
- Cà phê sữa
- Cà phê kem tươi
- Latte đường nâu
- Latte kẹo đường
Mong bạn hài lòng với câu trả lời của chúng tôi.
Hiện tại, chúng tôi chỉ trả lời các câu hỏi liên quan đến thông tin sản phẩm, sản phẩm bán chạy và gợi ý sản phẩm.
Ví dụ:
- Cà phê sữa còn hàng không?
- Cho tôi danh sách sản phẩm bán chạy của quán.
- Gợi ý cà phê tốt nhất.
Chân thành cảm ơn bạn đã sử dụng dịch vụ của quán.


In [ ]:
print(chatBotGetMessage("gợi ý cà phê tốt nhất", "KH0001"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━

In [ ]:
!pip install flask
!pip install fastapi
!pip install uvicorn
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import asyncio

In [ ]:
ngrok.set_auth_token("2pniti2Qbwn25ecH3KPLHlefUXi_EVwbfsrrgWfJS4KScpdN")

In [ ]:
# Request model
class ChatRequest(BaseModel):
    question: str
    user_id: str

# FastAPI app
app = FastAPI()

@app.post("/chat")
def get_response(request: ChatRequest):
    response = chatBotGetMessage(request.question, request.user_id)
    return {"response": response}

In [ ]:
# Open a tunnel on port 8000 for public access
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Configure uvicorn to use the existing event loop
config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)

# Start the server using the existing event loop
# asyncio.run() is not needed in Jupyter environments, instead use:
loop = asyncio.get_event_loop()
loop.create_task(server.serve())

# Or: asyncio.ensure_future(server.serve())

Public URL: NgrokTunnel: "https://b43e-35-233-182-87.ngrok-free.app" -> "http://localhost:8000"


<Task pending name='Task-1' coro=<Server.serve() running at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:68>>